In [1]:
from sklearnex import patch_sklearn

patch_sklearn()

import numpy as np
import scipy.io
import os

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.covariance import LedoitWolf

from pyriemann.estimation import Covariances
from pyriemann.classification import TSclassifier
from pyriemann.spatialfilters import CSP
from pyriemann.tangentspace import TangentSpace
#from pyriemann.classification import SVC

from sklearn.svm import SVC

import h5py

import time

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
start_time = time.time()

In [3]:
LoadingPath_Non='/home/nicole/Documents/AudioCueWalking_analysis/Variables/AdaptVsNon/LOSO_CV/Dataset/Ns_100/Delay'
LoadingPath_PS = '/home/nicole/Documents/AudioCueWalking_analysis/Variables/AdaptVsNon/LOSO_CV/Dataset/Ns_100/Delay/Transformed/IndividualSubjects_Adaptive/TrainingTestingDatasets'


#PreAlignmentCondition = 'Yes'
subject_ids = [0, 1] 
#subject_ids = [9, 10, 11, 12, 13, 14, 15, 16, 17] 

In [4]:
Nc = 108

# Load datasets

In [5]:
import os
import h5py
import numpy as np
import scipy.io

def load_subject_data(loading_path, subject_id, n_components=108, PreAlignment='No'):
    """
    Load (raw or prealigned) training and testing datasets, projected datasets, and features for a given subject.
    
    Parameters:
        loading_path (str): Path to the directory containing the dataset files.
        subject_id (int): Subject ID (0-based index).
        n_components (int): Number of components/features to extract.
        
    Returns:
        dict: A dictionary containing all loaded datasets and features.
    """
    # File paths
    if  PreAlignment == 'No':
        training_dataset_path = os.path.join(loading_path, 'TrainingDataset.mat')
        testing_dataset_path = os.path.join(loading_path, 'TestingDataset.mat') 
        training_projected_path = os.path.join(loading_path, 'TrainingProjected_sub001_002.mat')
        testing_projected_path = os.path.join(loading_path, 'TestingProjected_sub001_002.mat')
        training_features_path = os.path.join(loading_path, 'TrainingFeatures.mat')
        testing_features_path = os.path.join(loading_path, 'TestingFeatures.mat')
    elif PreAlignment == 'Yes':
        training_dataset_path = os.path.join(loading_path, 'TrainingDataset_sub001_002.mat')
        testing_dataset_path = os.path.join(loading_path, 'TestingDataset.mat') 
        training_projected_path = os.path.join(loading_path, 'TrainingProjected_sub001_002.mat')
        testing_projected_path = os.path.join(loading_path, 'TestingProjected_sub001_002.mat')
        training_features_path = os.path.join(loading_path, 'TrainingFeatures_sub001_002.mat')
        testing_features_path = os.path.join(loading_path, 'TestingFeatures_sub001_002.mat')

    # Load Training Dataset
    with h5py.File(training_dataset_path, 'r') as f_training_dataset:
        training_dataset = f_training_dataset['TrainingDataset']
        subject_cell_ref = training_dataset[subject_id, 0]
        subject_data = f_training_dataset[subject_cell_ref]
        X_train_dataset = np.transpose(subject_data['x'][:], (0, 1, 2))
        Y_train_dataset = np.squeeze(subject_data['y'][:])

    # Load Testing Dataset
    if  PreAlignment == 'No':
        # Advance
        # f_testing_dataset = scipy.io.loadmat(testing_dataset_path)
        # testing_dataset = f_testing_dataset['TestingDataset'][0, subject_id]
        # X_test_dataset = np.transpose(np.array(testing_dataset['x'][0][0]), (2, 1, 0))
        # #X_test_dataset = np.abs(X_test_dataset) #Added pre-aligment data is complex
        # Y_test_dataset = np.squeeze(np.array(testing_dataset['y'][0][0]))
        
        #Delay
        with h5py.File(testing_dataset_path, 'r') as f_testing_dataset:
            testing_dataset = f_testing_dataset['TestingDataset']
            subject_cell_ref = testing_dataset[subject_id, 0]
            subject_data = f_testing_dataset[subject_cell_ref]
            X_test_dataset = np.transpose(subject_data['x'][:], (0, 1, 2))
            Y_test_dataset = np.squeeze(subject_data['y'][:])

    elif PreAlignment == 'Yes':
        with h5py.File(testing_dataset_path, 'r') as f_testing_dataset:
            testing_dataset = f_testing_dataset['TestingDataset']
            subject_cell_ref = testing_dataset[subject_id, 0]
            subject_data = f_testing_dataset[subject_cell_ref]
            X_test_dataset = np.transpose(subject_data['x'][:], (0, 1, 2))
            Y_test_dataset = np.squeeze(subject_data['y'][:])


    # Load Training Projected Dataset
    with h5py.File(training_projected_path, 'r') as f_training_projected:
        training_projected = f_training_projected['TrainingProjected']
        subject_cell_ref = training_projected[subject_id, 0]
        subject_data = f_training_projected[subject_cell_ref]
        X_train_projected = np.transpose(subject_data['x'][:], (0, 2, 1))
        Y_train_projected = np.squeeze(subject_data['y'][:])

    # Load Testing Projected Dataset
    if  PreAlignment == 'No':
        #Advance
        # f_testing_projected = scipy.io.loadmat(testing_projected_path)
        # testing_projected = f_testing_projected['TestingProjected'][0, subject_id]
        # X_test_projected = np.transpose(np.array(testing_projected['x'][0][0]), (2, 0, 1))
        # #X_test_projected = np.abs(X_test_projected) #Added pre-aligment data is complex
        # Y_test_projected = np.squeeze(np.array(testing_projected['y'][0][0]))
        
        #Delay
        with h5py.File(testing_projected_path, 'r') as f_testing_projected:
            testing_projected = f_testing_projected['TestingProjected']
            subject_cell_ref = testing_projected[subject_id, 0]
            subject_data = f_testing_projected[subject_cell_ref]
            X_test_projected = np.transpose(subject_data['x'][:], (0, 2, 1))
            Y_test_projected = np.squeeze(subject_data['y'][:])
            
    elif PreAlignment == 'Yes':
        with h5py.File(testing_projected_path, 'r') as f_testing_projected:
            testing_projected = f_testing_projected['TestingProjected']
            subject_cell_ref = testing_projected[subject_id, 0]
            subject_data = f_testing_projected[subject_cell_ref]
            X_test_projected = np.transpose(subject_data['x'][:], (0, 2, 1))
            Y_test_projected = np.squeeze(subject_data['y'][:])

    # Load Training Features
    f_training_features = scipy.io.loadmat(training_features_path)
    training_features = f_training_features['TrainingFeatures'][0, subject_id]
    X_train_features = training_features[:, :n_components]
    Y_train_features = training_features[:, n_components:].flatten()

    # Load Testing Features
    f_testing_features = scipy.io.loadmat(testing_features_path)
    testing_features = f_testing_features['TestingFeatures'][0, subject_id]
    X_test_features = testing_features[:, :n_components]
    Y_test_features = testing_features[:, n_components:].flatten()

    # Organize results into a dictionary
    data = {
        "X_train_dataset": X_train_dataset,
        "Y_train_dataset": Y_train_dataset,
        "X_test_dataset": X_test_dataset,
        "Y_test_dataset": Y_test_dataset,
        "X_train_projected": X_train_projected,
        "Y_train_projected": Y_train_projected,
        "X_test_projected": X_test_projected,
        "Y_test_projected": Y_test_projected,
        "X_train_features": X_train_features,
        "Y_train_features": Y_train_features,
        "X_test_features": X_test_features,
        "Y_test_features": Y_test_features,
    }

    return data


# Classification Algorithms 

## Riemannian Geometry

In [6]:
def RiemannianGeometry(X_train_dataset, Y_train_dataset, X_test_dataset, Y_test_dataset, metric='riemann', C=1.0, kernel='linear'):
    """
    Projects dataset onto TS and classifies

    Parameters:
        X_train (array): Training dataset features.
        Y_train (array): Training dataset labels.
        X_test (array): Testing dataset features.
        Y_test (array): Testing dataset labels.
        metric (str): Metric for the TSclassifier (default: 'riemann').
        C (float): Regularization parameter for the SVM (default: 1.0).
        kernel (str): Kernel type for the SVM (default: 'linear').

    Returns:
        dict: A dictionary containing the predicted labels and performance metrics (accuracy, recall, precision, F1 score).
    """
    # Initialize the covariance estimator
    cov_estimator = Covariances(estimator='lwf')  # Regularized covariance estimation

    
    # Transform training and testing datasets
    X_train_cov = cov_estimator.transform(X_train_dataset)
    X_test_cov = cov_estimator.transform(X_test_dataset)
    
    # Initialize the TSclassifier with an SVC
    clf_TS = TSclassifier(metric=metric, clf=SVC(C=C, kernel=kernel))
    
    # Train the classifier
    clf_TS.fit(X_train_cov, Y_train_dataset)
    
    # Make predictions on the test set
    y_pred = clf_TS.predict(X_test_cov)
    
    # Calculate performance metrics
    accuracy = accuracy_score(Y_test_dataset, y_pred) 
    recall = recall_score(Y_test_dataset, y_pred) 
    precision = precision_score(Y_test_dataset, y_pred) 
    f1 = f1_score(Y_test_dataset, y_pred) 

    # Print metrics
    print(f"RieGeo A: {accuracy:.4f}%")
    print(f"RieGeo R: {recall:.4f}%")
    print(f"RieGeo P: {precision:.4f}%")
    print(f"RieGeo F1: {f1:.4f}%")
    
    # Organize results into a dictionary
    metrics = {
        "accuracy": accuracy,
        "recall": recall,
        "precision": precision,
        "f1_score": f1,
        #"predictions": y_pred,
    }
    
    return metrics

## CSP: Arithmetic average "Spatial filtered (A) + Riemann"
### Project spatially filtered signals (ProjectedDataset) to tangent space 

In [7]:
def SpatFilt_A_Rie(X_train_projected, Y_train_projected, X_test_projected, Y_test_dataset, metric='riemann', C=1.0, kernel='linear'):
    """
    Projects spatially filtered (A) features onto TS and classifies
    
    Parameters:
        X_train_projected (np.ndarray): Training data (projected).
        Y_train_projected (np.ndarray): Training labels (projected).
        X_test_projected (np.ndarray): Testing data (projected).
        Y_test_dataset (np.ndarray): True labels for the testing data.
        metric (str): Metric to use for the TSclassifier. Default is 'riemann'.
        C (float): Regularization parameter for the SVM. Default is 1.0.
        kernel (str): Kernel type for the SVM. Default is 'linear'.
    
    Returns:
        dict: A dictionary containing accuracy, recall, precision, and F1 score.
    """
    # Estimate covariance matrices
    cov_estimator_projected = Covariances(estimator='lwf')  # Helps with regularization
    X_train_cov_projected = cov_estimator_projected.transform(X_train_projected)
    X_test_cov_projected = cov_estimator_projected.transform(X_test_projected)

    # Train the classifier
    clf_TS_proj = TSclassifier(metric=metric, clf=SVC(C=C, kernel=kernel, class_weight='balanced'))
    clf_TS_proj.fit(X_train_cov_projected, Y_train_projected)

    # Make predictions on the test set
    y_pred_proj = clf_TS_proj.predict(X_test_cov_projected)
    
    # Compute performance metrics
    accuracy = accuracy_score(Y_test_dataset, y_pred_proj) 
    recall = recall_score(Y_test_dataset, y_pred_proj) 
    precision = precision_score(Y_test_dataset, y_pred_proj) 
    f1 = f1_score(Y_test_dataset, y_pred_proj) 
    
    # Print metrics
    print(f"SpatFilt(A) + Rie A: {accuracy:.4f}%")
    print(f"SpatFilt(A) + Rie R: {recall:.4f}%")
    print(f"SpatFilt(A) + Rie P: {precision:.4f}%")
    print(f"SpatFilt(A) + Rie F1: {f1:.4f}%")

    # Return metrics as a dictionary
    metrics = {
        "accuracy": accuracy,
        "recall": recall,
        "precision": precision,
        "f1_score": f1,
    }
    return metrics

## CSP: Riemannian average (1) "CSP(R) + Riemann"
### Use pyriemann.spatialfilters.csp (TrainingDataset) to obtain spatially filtered signals, project and classify in tangent space

In [8]:
def CSP_R_Rie(X_train_dataset, Y_train_dataset, X_test_dataset, Y_test_dataset, 
                           nfilter=108, metric='riemann', log=False, C=1.0, kernel='linear'):
    """
    Obtains CSP (using riemannian mean), spatially filters dataset, projects onto TS and classifies
    
    Parameters:
        X_train_dataset (np.ndarray): Training data (raw).
        Y_train_dataset (np.ndarray): Training labels.
        X_test_dataset (np.ndarray): Testing data (raw).
        Y_test_dataset (np.ndarray): True labels for the testing data.
        nfilter (int): Number of filters for the CSP. Default is 4.
        metric (str): Metric to use for the TSclassifier. Default is 'riemann'.
        log (bool): Whether to apply log transformation in CSP. Default is False.
        C (float): Regularization parameter for the SVM. Default is 1.0.
        kernel (str): Kernel type for the SVM. Default is 'linear'.
    
    Returns:
        dict: A dictionary containing accuracy, recall, precision, and F1 score.
    """
    # Covariance matrix estimation
    cov_estimator = Covariances(estimator='lwf')
    X_train_cov = cov_estimator.transform(X_train_dataset)
    X_test_cov = cov_estimator.transform(X_test_dataset)

    # CSP transformation
    csp = CSP(nfilter=nfilter, metric=metric, log=log)
    X_train_csp = csp.fit_transform(X_train_cov, Y_train_dataset)
    X_test_csp = csp.transform(X_test_cov)

    # Train the classifier
    clf_TS_features = TSclassifier(metric=metric, clf=SVC(C=C, kernel=kernel, class_weight='balanced'))
    clf_TS_features.fit(X_train_csp, Y_train_dataset)

    # Make predictions on the test set
    y_pred_features = clf_TS_features.predict(X_test_csp)
    
    
    # Compute performance metrics
    accuracy = accuracy_score(Y_test_dataset, y_pred_features) 
    recall = recall_score(Y_test_dataset, y_pred_features) 
    precision = precision_score(Y_test_dataset, y_pred_features) 
    f1 = f1_score(Y_test_dataset, y_pred_features) 

    # Print metrics
    print(f"CSP(R) + Rie A: {accuracy:.4f}%")
    print(f"CSP(R) + Rie R: {recall:.4f}%")
    print(f"CSP(R) + Rie P: {precision:.4f}%")
    print(f"CSP(R) + Rie F1: {f1:.4f}%")

    # Return metrics as a dictionary
    metrics = {
        "accuracy": accuracy,
        "recall": recall,
        "precision": precision,
        "f1_score": f1,
    }
    
    return metrics

## CSP: Concatenation of features 
### Concatenate CSP variance features and tangent space features 

In [9]:
def Concat(X_train_dataset, Y_train_dataset, X_test_dataset, Y_test_dataset, 
                                    X_train_features, X_test_features, metric='riemann', C=1.0, kernel='linear'):
    """
    Concatenates spatially filtered features and features found from TS projection of dataset and classfies
    
    Parameters:
        X_train_dataset (np.ndarray): Training data (raw).
        Y_train_dataset (np.ndarray): Training labels.
        X_test_dataset (np.ndarray): Testing data (raw).
        Y_test_dataset (np.ndarray): True labels for the testing data.
        X_train_features (np.ndarray): Additional training features.
        X_test_features (np.ndarray): Additional testing features.
        metric (str): Metric to use for TangentSpace. Default is 'riemann'.
        C (float): Regularization parameter for the SVM. Default is 1.0.
        kernel (str): Kernel type for the SVM. Default is 'linear'.
    
    Returns:
        dict: A dictionary containing accuracy, recall, precision, and F1 score.
    """
    
    # --- Find tanget space features ---
    # Covariance matrix estimation
    cov_estimator = Covariances(estimator='lwf')  # Helps with regularization
    X_train_cov = cov_estimator.transform(X_train_dataset)
    X_test_cov = cov_estimator.transform(X_test_dataset)

    # Tangent Space transformation
    TSpace = TangentSpace(metric=metric, tsupdate=False).fit(X_train_cov)
    X_train_TS = TSpace.transform(X_train_cov)
    X_test_TS = TSpace.transform(X_test_cov)
    
    # --- Concatenate ---
    # Combine features and Tangent Space data
    X_train_cat = np.concatenate((X_train_features, X_train_TS), axis=1)
    X_test_cat = np.concatenate((X_test_features, X_test_TS), axis=1)

    # Train the classifier
    clf_TS_cat = SVC(C=C, kernel=kernel, class_weight='balanced')
    clf_TS_cat.fit(X_train_cat, Y_train_dataset)

    # Make predictions on the test set
    y_pred_cat = clf_TS_cat.predict(X_test_cat)
    
    # Compute performance metrics
    accuracy = accuracy_score(Y_test_dataset, y_pred_cat) 
    recall = recall_score(Y_test_dataset,y_pred_cat) 
    precision = precision_score(Y_test_dataset, y_pred_cat) 
    f1 = f1_score(Y_test_dataset, y_pred_cat) 

    # Print metrics
    print(f"Concat A: {accuracy:.4f}%")
    print(f"Concat R: {recall:.4f}%")
    print(f"Concat P: {precision:.4f}%")
    print(f"Concat F1: {f1:.4f}%")

    
    # Return metrics as a dictionary
    metrics = {
        "accuracy": accuracy,
        "recall": recall,
        "precision": precision,
        "f1_score": f1,
    }
    
    return metrics

# Implement functions

In [ ]:
Results = {}

# loading_paths = [
#     ("LoadingPath_Non", "No"), 
#     ("LoadingPath_PS", "Yes")  
# ]

 
loading_paths = [
    (LoadingPath_PS, "Yes")  
]

for loading_path, pre_alignment_condition in loading_paths:
    print(f"Processing {loading_path} with PreAlignmentCondition = {pre_alignment_condition}")
    
    # Loop through each subject
    for subject_id in [0]:#subject_ids:
        print(f"Loading data for Subject {subject_id}...")
        
        # Load the subject data
        subject_data = load_subject_data(loading_path, subject_id, n_components=Nc, PreAlignment=pre_alignment_condition)
        
        # Dataset
        X_train_dataset = subject_data["X_train_dataset"]
        Y_train_dataset = subject_data["Y_train_dataset"]
        X_test_dataset = subject_data["X_test_dataset"]
        Y_test_dataset = subject_data["Y_test_dataset"]
        
        # Projected
        X_train_projected = subject_data["X_train_projected"]
        Y_train_projected = subject_data["Y_train_projected"]
        X_test_projected = subject_data["X_test_projected"]
        Y_test_projected = subject_data["Y_test_projected"]
        
        # Features
        X_train_features = subject_data["X_train_features"]
        Y_train_features = subject_data["Y_train_features"]
        X_test_features = subject_data["X_test_features"]
        Y_test_features = subject_data["Y_test_features"]
        
        # Sanity check
        print(X_train_dataset.shape)
        print(Y_train_dataset.shape)
        print(X_test_dataset.shape)
        print(Y_test_dataset.shape)

        print(X_train_projected.shape) 
        print(Y_train_projected.shape)
        print(X_test_projected.shape) 
        print(Y_test_projected.shape)

        print(X_train_features.shape) 
        print(Y_train_features.shape)
        print(X_test_features.shape)
        print(Y_test_features.shape)
        
        # Implement classification methods
        #RG_Metrics = RiemannianGeometry(X_train_dataset, Y_train_dataset, X_test_dataset, Y_test_dataset, metric='riemann', C=1.0, kernel='linear')
        #SpatFilt_A_Rie_Metrics = SpatFilt_A_Rie(X_train_projected, Y_train_projected, X_test_projected, Y_test_dataset, metric='riemann', C=1.0, kernel='linear')
        #CSP_R_Rie_Metrics = CSP_R_Rie(X_train_dataset, Y_train_dataset, X_test_dataset, Y_test_dataset, nfilter=Nc, metric='riemann', log=False, C=1.0, kernel='linear')
        Concat_Metrics = Concat(X_train_dataset, Y_train_dataset, X_test_dataset, Y_test_dataset, X_train_features, X_test_features, metric='riemann', C=1.0, kernel='linear')
        
        # Store the results for each subject
        if subject_id not in Results:
            Results[subject_id] = {}
        
        Results[subject_id][f"{loading_path}_SpatFilt_A_Rie_Metrics"] = SpatFilt_A_Rie_Metrics
        Results[subject_id][f"{loading_path}_Concat_Metrics"] = Concat_Metrics
        # Results[subject_id][f"{loading_path}_RG_Metrics"] = RG_Metrics
        # Results[subject_id][f"{loading_path}_CSP_R_Rie_Metrics"] = CSP_R_Rie_Metrics
        
        # Clear variables to free memory
        del (
            X_train_dataset, Y_train_dataset,
            X_test_dataset, Y_test_dataset,
            X_train_projected, Y_train_projected,
            X_test_projected, Y_test_projected,
            X_train_features, Y_train_features,
            X_test_features, Y_test_features
        )
        
        print(f"Cleared data for Subject {subject_id}.")
    
print(Results)

Processing /home/nicole/Documents/AudioCueWalking_analysis/Variables/AdaptVsNon/LOSO_CV/Dataset/Ns_100/Delay/Transformed/IndividualSubjects_Adaptive/TrainingTestingDatasets with PreAlignmentCondition = Yes
Loading data for Subject 0...
(50538, 108, 100)
(50538,)
(3278, 108, 100)
(3278,)
(50538, 108, 100)
(50538,)
(3278, 108, 100)
(3278,)
(50538, 108)
(50538,)
(3278, 108)
(3278,)


In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time/60, "minutes")

In [ ]:
Results[0]

# Finish

# Subset testing